    16_230109_100_kyungki_apt_안양
    
     
      data : apt_안양_여주_531.xls
     save : apt_anyang.csv
     display : apt_anyang.html

In [1]:
#1. package load
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap
print('슝~')

슝~


In [2]:
#2. data load  ...SGIS data 가져오기 # SGIS API 사이트의 본인의 consumer_key, consumer_secret를 입력합니다.
consumer_key = '2f74504a63224ee09f2b'       # 서비스ID
consumer_secret =  '63a0ffef893e43a48ae2' # 보안Key


In [3]:
#2. data load  ... SGIS data 가져오기 # SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
   
    return access_token
print('슝~')

슝~


In [4]:
#9 Folium를 활용한 공간 데이터 시각화
### 전국도시철도역사정보표준데이터 가져오기

apt_df = pd.read_excel("/aiffel/aiffel/000_aiffelthon_230111/3김연수/002_data_dir/002_apt_data/apt_안양_여주_531.xls")
apt_df.head()

,시군명,공동주택구분,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부
0,안양시,NaN,DY town,호계3동,971-1,2013-01-03,9,2,115,N
1,안양시,NaN,HHI브라운빌3차아파트,호계2동,940,2007-07-27,15,2,149,N
2,안양시,NaN,LD범계마을아파트,호계2동,923-30,1999-01-13,17,1,113,N
3,안양시,NaN,LG빌리지,석수2동,484,2001-05-19,10,34,1872,Y
4,안양시,NaN,강남6차아파트,비산2동,566-5,1984-10-25,6,1,54,N


In [5]:
apt_df.drop(["공동주택구분"], axis=1,inplace=True)
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부
0,안양시,DY town,호계3동,971-1,2013-01-03,9,2,115,N
1,안양시,HHI브라운빌3차아파트,호계2동,940,2007-07-27,15,2,149,N
2,안양시,LD범계마을아파트,호계2동,923-30,1999-01-13,17,1,113,N
3,안양시,LG빌리지,석수2동,484,2001-05-19,10,34,1872,Y
4,안양시,강남6차아파트,비산2동,566-5,1984-10-25,6,1,54,N


In [6]:
apt_df['지번주소'] = apt_df['지번주소'].astype(str)

In [7]:
#okok
apt_df['addr'] = '경기도 ' + apt_df[['시군명', '읍면동주소','지번주소']].agg(' '.join, axis=1)
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr
0,안양시,DY town,호계3동,971-1,2013-01-03,9,2,115,N,경기도 안양시 호계3동 971-1
1,안양시,HHI브라운빌3차아파트,호계2동,940,2007-07-27,15,2,149,N,경기도 안양시 호계2동 940
2,안양시,LD범계마을아파트,호계2동,923-30,1999-01-13,17,1,113,N,경기도 안양시 호계2동 923-30
3,안양시,LG빌리지,석수2동,484,2001-05-19,10,34,1872,Y,경기도 안양시 석수2동 484
4,안양시,강남6차아파트,비산2동,566-5,1984-10-25,6,1,54,N,경기도 안양시 비산2동 566-5


In [8]:
# https://sgisapi.kostat.go.kr/OpenAPI3/addr/stage.json

In [9]:
#5 seoul university   # 주소를 입력해 주소의 위도, 경도를 가지고 오는 함수
def get_adress_x_y(address) -> None:
    # SGIS access_token 가져오기
    access_token = get_access_token(consumer_key , consumer_secret)

    url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params={
        "accessToken": access_token,
        "address": address
    }

    temp_res = requests.get(
        url=url,
        params=params
    ).json()["result"]["resultdata"]

    return temp_res[0]["x"], temp_res[0]["y"]

print('슝~')

슝~


In [10]:
#5 seoul university .. 주소 기반 위도,경도 추가
cnt = 0
for _, row in apt_df.iterrows():
    try:
        x, y = get_adress_x_y(row["addr"])
     
        apt_df.loc[_, "x"] = x
        apt_df.loc[_, "y"] = y
    except:
        print(row["addr"] + '은(는) 검색에 실패했습니다')
        cnt = cnt + 1

print("실패건수 : ", cnt)
apt_df.head()


경기도 안양시 박달1동 149-12은(는) 검색에 실패했습니다
경기도 안양시 박달1동 13241은(는) 검색에 실패했습니다
경기도 안양시 박달1동 44900은(는) 검색에 실패했습니다
경기도 안양시 호계1동 966-1은(는) 검색에 실패했습니다
경기도 안양시 안양6동 423-3은(는) 검색에 실패했습니다
경기도 안양시 안양1동 32874은(는) 검색에 실패했습니다
경기도 안양시 석수3동 346-3은(는) 검색에 실패했습니다
경기도 안양시 석수2동 864은(는) 검색에 실패했습니다
경기도 안양시 안양2동 15008은(는) 검색에 실패했습니다
경기도 안양시 안양9동 10601060-2은(는) 검색에 실패했습니다
경기도 안양시 안양2동 16103은(는) 검색에 실패했습니다
경기도 안양시 안양2동 44925은(는) 검색에 실패했습니다
경기도 안양시 안양7동 1410은(는) 검색에 실패했습니다
경기도 안양시 안양2동 11689은(는) 검색에 실패했습니다
경기도 안양시 안양7동 433은(는) 검색에 실패했습니다
경기도 안양시 안양1동 29312은(는) 검색에 실패했습니다
경기도 안양시 안양6동 552-9은(는) 검색에 실패했습니다
경기도 안양시 평촌동 33970은(는) 검색에 실패했습니다
경기도 안양시 평촌동 27426은(는) 검색에 실패했습니다
경기도 안양시 평촌동 27395은(는) 검색에 실패했습니다
경기도 안양시 호계1동 1296은(는) 검색에 실패했습니다
경기도 안양시 비산2동 1166은(는) 검색에 실패했습니다
경기도 양주시 백석읍 복지리 34029은(는) 검색에 실패했습니다
경기도 양주시 덕정동 산37-31은(는) 검색에 실패했습니다
경기도 양주시 백석읍 복지리 36192은(는) 검색에 실패했습니다
경기도 양주시 광적면 가납리 산54-7은(는) 검색에 실패했습니다
경기도 양주시 백석읍 오산리 12055은(는) 검색에 실패했습니다
경기도 양주시 백석읍 가업리 36161은(는) 검색에 실패했습니다
경기도 양주시 백석읍 복지리 14977은(는) 검색에 실패했습니다


,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr,x,y
0,안양시,DY town,호계3동,971-1,2013-01-03,9,2,115,N,경기도 안양시 호계3동 971-1,951780.982195556,1930038.46615583
1,안양시,HHI브라운빌3차아파트,호계2동,940,2007-07-27,15,2,149,N,경기도 안양시 호계2동 940,951648.800972895,1930981.84433521
2,안양시,LD범계마을아파트,호계2동,923-30,1999-01-13,17,1,113,N,경기도 안양시 호계2동 923-30,951221.572580647,1931672.97443054
3,안양시,LG빌리지,석수2동,484,2001-05-19,10,34,1872,Y,경기도 안양시 석수2동 484,947025.809510663,1936663.26786704
4,안양시,강남6차아파트,비산2동,566-5,1984-10-25,6,1,54,N,경기도 안양시 비산2동 566-5,950089.066159404,1933251.2933193


In [11]:
#5 seoul university  ..# SGIS 좌표변환 API를 활용한 좌표변환 함수
def get_transcoord(src, dst, posX, posY) -> None:
    access_token = get_access_token(consumer_key, consumer_secret)
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"] # 변환된 좌표값 딕셔너리의 value 리턴

print('슝~')


슝~


In [12]:
#5 seoul university  .. 좌표계가 UTM-K이므로 WSG84좌표계로 변환
for _, row in apt_df.iterrows():
    try:
        x, y = get_transcoord("EPSG:5179", "EPSG:4326", row["x"], row["y"])
    
        apt_df.loc[_, "lng"] = x
        apt_df.loc[_, "lat"] = y

    except:
        pass
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr,x,y,lng,lat
0,안양시,DY town,호계3동,971-1,2013-01-03,9,2,115,N,경기도 안양시 호계3동 971-1,951780.982195556,1930038.46615583,126.955433,37.368156
1,안양시,HHI브라운빌3차아파트,호계2동,940,2007-07-27,15,2,149,N,경기도 안양시 호계2동 940,951648.800972895,1930981.84433521,126.953879,37.376652
2,안양시,LD범계마을아파트,호계2동,923-30,1999-01-13,17,1,113,N,경기도 안양시 호계2동 923-30,951221.572580647,1931672.97443054,126.949008,37.382859
3,안양시,LG빌리지,석수2동,484,2001-05-19,10,34,1872,Y,경기도 안양시 석수2동 484,947025.809510663,1936663.26786704,126.901258,37.427608
4,안양시,강남6차아파트,비산2동,566-5,1984-10-25,6,1,54,N,경기도 안양시 비산2동 566-5,950089.066159404,1933251.2933193,126.936109,37.397025


In [13]:
#5 seoul university  .. 시각화 .. 초기 중심값 좌표가 들어가야 합니다.

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 12로 시작합니다.
map = folium.Map(
    location=center,
    zoom_start=12

)

print('슝~')

슝~


In [15]:
#2. 프로젝트(1)  # MarkerCluster 객체 
marker_cluster = MarkerCluster().add_to(map)

#maker 객체 추가
for idx, row in apt_df.iterrows():
    try:
        folium.Marker(
            location=[apt_df.loc[idx, "lat"], apt_df.loc[idx, "lng"]],
            popup=apt_df.loc[idx, "공동주택명정보"],
            icon=folium.Icon(color="cadetblue", icon="house")            
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

슝~


In [16]:
#5 seoul university 
map

In [17]:
map.save("/aiffel/aiffel/000_aiffelthon_230111/3김연수/003_map_xml/apt_anyang.html")

In [18]:
apt_df.to_csv("apt_anyang.csv", index = False)